In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as gemeni
from pinecone import Pinecone, ServerlessSpec

In [18]:
gemeni.configure(api_key=os.environ["GEMENI_API_KEY"])
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

pc.delete_index("rag")

pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [19]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Introduction to Computer Science',
  'stars': 5,
  'review': 'Dr. Johnson explains concepts clearly and makes difficult topics easier to understand. Great lectures!'},
 {'professor': 'Prof. Michael Smith',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Good professor, but the pace of the class can be overwhelming at times. Weekly quizzes are helpful though.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Modern Literature',
  'stars': 5,
  'review': 'One of the best professors! She brings literature to life, and her classes are always engaging.'},
 {'professor': 'Dr. Robert Kim',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': 'Very difficult course. Dr. Kim is knowledgeable, but the exams are extremely tough, and explanations could be clearer.'},
 {'professor': 'Prof. Jessica Brown',
  'subject': 'World History',
  'stars': 4,
  'review': 'Her lectures are thorough, and she is passionate about history. Workload is a

In [20]:
processed_data = []
model_name = "models/embedding-gecko-001"
# model = genai.GenerativeModel('gemini-1.5-flash')

for review in data['reviews']:
    response = gemeni.generate_embeddings(
        model=model_name,
        text=review['review'],
    )
    embedding = response['embedding']
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [ ]:
index=pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ms1"
)